# ConversationBufferMemory

In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.clear()

memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})

{'history': 'Human: Hello chatbot!\nAI: Hello. How can I help you?\nHuman: My name is Terry\nAI: Nice to meet you Terry\nHuman: Where is Seoul?\nAI: Seoul is in Korea'}

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
memory.clear()
memory.chat_memory.add_user_message("Hello chatbot!")
memory.chat_memory.add_ai_message("Hello. How can I help you?")
memory.load_memory_variables({})

{'chat_history': 'Human: Hello chatbot!\nAI: Hello. How can I help you?'}

In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})
# returning list of chat message

{'chat_history': [HumanMessage(content='Hello chatbot!'),
  AIMessage(content='Hello. How can I help you?'),
  HumanMessage(content='My name is Terry'),
  AIMessage(content='Nice to meet you Terry'),
  HumanMessage(content='Where is Seoul?'),
  AIMessage(content='Seoul is in Korea')]}

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})
# returning chat history with single string

{'chat_history': 'Human: Hello chatbot!\nAI: Hello. How can I help you?\nHuman: My name is Terry\nAI: Nice to meet you Terry\nHuman: Where is Seoul?\nAI: Seoul is in Korea'}

# Conversational Buffer Memory for LLM

In [6]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "{YOUR_LANGCHAIN_APIKEY}"
os.environ["LANGCHAIN_PROJECT"] = "{YOUR_PROJECT_NAME}"

model = OpenAI(openai_api_key="{YOUR_OPENAI_KEY}", temperature=0)

template = """You are a chatbot having a conversation with a human.

Previous conversation history:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(llm=model, prompt=prompt, verbose=True, memory=memory)

In [7]:
conversation.invoke({"question": "hi my name is Terry"})
conversation.invoke({"question": "Can you recommend fun activities for me?"})
conversation.invoke({"question": "What is my name?"})
memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Previous conversation history:


New human question: hi my name is Terry
Response:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Previous conversation history:
Human: hi my name is Terry
AI:  Hello Terry, it's nice to meet you! How can I assist you today?

New human question: Can you recommend fun activities for me?
Response:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Previous conversation history:
Human: hi my name is Terry
AI:  Hello Terry, it's nice to meet you! How can I assist you today?
Human: Can you recommend fun activities for me?
AI:  Of course! There are many fun activities you can try, depending on your interests. If you enjoy being outdoors, you could go for a hike or have a picnic in

{'chat_history': "Human: hi my name is Terry\nAI:  Hello Terry, it's nice to meet you! How can I assist you today?\nHuman: Can you recommend fun activities for me?\nAI:  Of course! There are many fun activities you can try, depending on your interests. If you enjoy being outdoors, you could go for a hike or have a picnic in the park. If you prefer indoor activities, you could try a new hobby like painting or cooking. You could also check out local events or festivals happening in your area. What type of activities do you enjoy?\nHuman: What is my name?\nAI:  Your name is Terry."}

# Conversational Buffer Memory for ChatLLM

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="{YOUR_LANGCHAIN_APIKEY}"
os.environ["LANGCHAIN_PROJECT"]="terrylangsmith"

llm = ChatOpenAI(openai_api_key="{YOUR_OPENAI_KEY}")
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear()
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [5]:
conversation.invoke({"question": "hi my name is Terry"})
conversation.invoke({"question": "Can you recommend fun activities for me?"})
conversation.invoke({"question": "What is my name?"})
memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: hi my name is Terry

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: hi my name is Terry
AI: Hello Terry! Nice to meet you. How can I assist you today?
Human: Can you recommend fun activities for me?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: hi my name is Terry
AI: Hello Terry! Nice to meet you. How can I assist you today?
Human: Can you recommend fun activities for me?
AI: Of course! I'd be happy to help. Could you let me know a bit more about your interests and preferences? That way, I can provide you with more tailored recommendations. Are you looking for indoor or outdoor activities? Do you have any particular hobbies or things you enjoy doing?
Human: Wh

{'chat_history': [HumanMessage(content='hi my name is Terry'),
  AIMessage(content='Hello Terry! Nice to meet you. How can I assist you today?'),
  HumanMessage(content='Can you recommend fun activities for me?'),
  AIMessage(content="Of course! I'd be happy to help. Could you let me know a bit more about your interests and preferences? That way, I can provide you with more tailored recommendations. Are you looking for indoor or outdoor activities? Do you have any particular hobbies or things you enjoy doing?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Terry. You mentioned it at the beginning of our conversation. Is there anything specific you would like assistance with, Terry?')]}



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice travel agent having a conversation with a human.
Human: hi my name is Terry
AI: Hello Terry! Nice to meet you. How can I assist you today? Are you looking to plan a trip or have any specific travel preferences?
Human: Can you recommend fun activities for me?

> Finished chain.


{'question': 'Can you recommend fun activities for me?',
 'chat_history': [HumanMessage(content='hi my name is Terry'),
  AIMessage(content='Hello Terry! Nice to meet you. How can I assist you today? Are you looking to plan a trip or have any specific travel preferences?'),
  HumanMessage(content='Can you recommend fun activities for me?'),
  AIMessage(content="Of course, Terry! I'd be happy to recommend some fun activities for you. Can you please let me know the destination you have in mind or any particular interests or preferences you have? That way, I can provide you with more tailored recommendations.")],
 'text': "Of course, Terry! I'd be happy to recommend some fun activities for you. Can you please let me know the destination you have in mind or any particular interests or preferences you have? That way, I can provide you with more tailored recommendations."}